In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [ ]:
def RotX(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[1, 1] = ct
    R[1, 2] = -st
    R[2, 1] = st
    R[2, 2] = ct
    return R

def RotY(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 2] = st
    R[2, 0] = -st
    R[2, 2] = ct
    return R

def RotZ(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 1] = -st
    R[1, 0] = st
    R[1, 1] = ct
    return R

In [ ]:
def hat(vec):
    v = vec.reshape((3,))
    return np.array([
        [0., -v[2], v[1]],
        [v[2], 0., -v[0]],
        [-v[1], v[0], 0.]
    ])

def unhat(mat):
    return np.array([[mat[2, 1], mat[0, 2], mat[1, 0]]]).T

def exp_rotation(p):
    phi = p.reshape((3, 1))
    theta = np.linalg.norm(phi)
    if theta < 1e-12:
        return np.eye(3, 3)
    a = phi / theta
    return np.eye(3) * np.cos(theta) + (1. - np.cos(theta)) * a @ a.T + np.sin(theta) * hat(a)

def log_rotation(R):
    theta = np.arccos(max(-1., min(1., (np.trace(R) - 1.) / 2.)))

    if np.isclose(theta, 0.):
        return np.zeros((3, 1))
    elif np.isclose(theta, np.pi):
        r00 = R[0, 0]
        r11 = R[1, 1]
        r22 = R[2, 2]

        r02 = R[0, 2]
        r12 = R[1, 2]

        r01 = R[0, 1]
        r21 = R[2, 1]

        r10 = R[1, 0]
        r20 = R[2, 0]

        if not np.isclose(r22, -1.):
            multiplier = theta / np.sqrt(2. * (1. + r22))
            return multiplier * np.array([[r02, r12, 1. + r22]]).T
        elif not np.isclose(r11, -1.):
            multiplier = theta / np.sqrt(2. * (1. + r11))
            return multiplier * np.array([[r01, 1. + r11, r21]]).T
        elif not np.isclose(r00, -1.):
            multiplier = theta / np.sqrt(2. * (1. + r00))
            return multiplier * np.array([[1. + r00, r10, r20]]).T
        else:
            print("ERROR: This should never happen!")
            exit(1)

    mat = R - R.T
    r = unhat(mat)

    return theta / (2. * np.sin(theta)) * r

def J_l(q, epsilon = 1e-8):
    n = np.linalg.norm(q)
    if n < epsilon:
        return np.eye(3)
    n_sq = n * n
    n_3 = n_sq * n
    c = np.cos(n)
    s = np.sin(n)
    hat_q = hat(q)
    hat_q_sq = hat_q @ hat_q
    JL = np.eye(3) + hat_q * ((1. - c) / n_sq) + hat_q_sq * ((n - s) / n_3)
    return JL

def J_l_inv(q, epsilon = 1e-8):
    n = np.linalg.norm(q)
    if n < epsilon:
        return np.eye(3)
    n_sq = n * n
    n_3 = n_sq * n
    c = np.cos(n)
    s = np.sin(n)
    hat_q = hat(q)
    hat_q_sq = hat_q @ hat_q
    JL_inv = np.eye(3) - 0.5 * hat_q + hat_q_sq * (1./n_sq - (1. + c) / (2. * n * s))
    return JL_inv

def exp_pose(tau):
    phi = tau[:3, :]
    rho = tau[3:, :]
    return np.block([[exp_rotation(phi), J_l(phi) @ rho], [0., 0., 0., 1.]])

def log_pose(T):
    R = T[:3, :3]
    t = T[:3, 3:]
    phi = log_rotation(R)
    return np.vstack([phi, -J_l_inv(phi) @ t])

np.set_printoptions(precision=2, suppress=True)

In [ ]:
def rotation_error_local(Rc, Rt):
    return Rc.T @ Rt

def rotation_error(Rc, Rt):
    return Rt @ Rc.T

# Let's test them
Rc = RotZ(1.)
Rt = RotZ(1.2)

# They should return the same here! Why?
print(rotation_error_local(Rc, Rt))
print("------------------------")
print(rotation_error(Rc, Rt))

print("========================")

Rc = RotZ(1.)
Rt = RotY(1.)

# They should return different here! Why?
print(rotation_error_local(Rc, Rt))
print("------------------------")
print(rotation_error(Rc, Rt))

print("========================")

In [ ]:
# Let's compute distances/errors with Lie algebra
phi_ct = log_rotation(Rc.T @ Rt)

print(phi_ct.T)
print(np.linalg.norm(phi_ct)**2, np.sum(phi_ct.T @ phi_ct))
print(np.linalg.norm(phi_ct), np.sqrt(np.sum(phi_ct.T @ phi_ct)))

In [ ]:
# Let's interpolate!
alpha = 0.7

R1 = exp_rotation(alpha * log_rotation(Rt @ Rc.T)) @ Rc
R2 = Rc @ exp_rotation(alpha * log_rotation(Rc.T @ Rt))

print(Rc)
print("=================")
print(Rt)
print("=================")
# These two should be the same!
print(R1)
print("=================")
print(R2)

In [ ]:
# Adjoint matrices
def adjoint_rotation(R):
    return R

def adjoint_pose(T):
    R = T[:3, :3]
    t = T[:3, 3:]
    return np.block([[R, np.zeros((3, 3))], [hat(t) @ R, R]])

In [ ]:
# Orientation integration!
# Angular Velocities
def angular_velocity_to_deriv_rotation(omega_w, R):
    return hat(omega_w) @ R

def angular_velocity_local_to_deriv_rotation(omega_b, R):
    return R @ hat(omega_b)

# Some testing
R = RotX(np.pi / 2.)

omega_b = np.array([[1., 0., 0.]]).T

# Integrate naively for a few steps
dt = 0.1
R1 = np.copy(R)
R2 = np.copy(R)
R3 = np.copy(R)
R4 = np.copy(R)

for i in range(50):
    omega_w = adjoint_rotation(R1) @ omega_b # rotate local omega to world frame with adjoint matrix!
    R1d = angular_velocity_to_deriv_rotation(omega_w, R1)
    R2d = angular_velocity_local_to_deriv_rotation(omega_b, R2)

    R1 = R1 + R1d * dt # Euler integration!
    R2 = R2 + R2d * dt # Euler integration!

    # SO(3) integration
    R3 = R3 @ exp_rotation(omega_b * dt)
    R4 = exp_rotation((adjoint_rotation(R4) @ omega_b) * dt) @ R4

# All should be very similar
print(R1)
print(np.linalg.det(R1))
print("========================")
print(R2)
print(np.linalg.det(R2))
print("========================")
print(R3)
print(np.linalg.det(R3))
print("========================")
print(R4)
print(np.linalg.det(R4))
print("========================")
print(np.linalg.norm(R1-R2), np.linalg.norm(R2-R3), np.linalg.norm(R3-R4))

In [ ]:
def skew_velocity(V):
    omega = V[:3, :]
    v  = V[3:, :]

    S = np.zeros(shape=(4, 4))
    S[:3, :3] = hat(omega)
    S[:3, 3:] = v

    return S

def spatial_velocity_to_deriv_homogeneous(Vw, T):
    return skew_velocity(Vw) @ T

def spatial_velocity_local_to_deriv_homogeneous(Vb, T):
    return T @ skew_velocity(Vb)

# Let's see if this is the same as pure Rotation kinematics
R = RotX(np.pi / 2.)

omega_b = np.array([[1., 0., 0.]]).T

T = np.block([[R, np.zeros((3, 1))], [0., 0., 0., 1.]])

Vb = np.vstack([omega_b, np.zeros((3, 1)) * 0.1])

# Integrate naively for a few steps
dt = 0.1
T1 = np.copy(T)
T2 = np.copy(T)
T3 = np.copy(T)
T4 = np.copy(T)

for i in range(50):
    Vw = adjoint_pose(T1) @ Vb # rotate local Vb to world space using the adjoint matrix!
    T1d = spatial_velocity_to_deriv_homogeneous(Vw, T1)
    T2d = spatial_velocity_local_to_deriv_homogeneous(Vb, T2)

    T1 = T1 + T1d * dt # Euler integration!
    T2 = T2 + T2d * dt # Euler integration!

    # SE(3) integration
    T3 = T3 @ exp_pose(Vb * dt)
    Vw = adjoint_pose(T4) @ Vb # rotate local Vb to world space using the adjoint matrix!
    T4 = exp_pose(Vw * dt) @ T4

# These two should be very similar
print(T1)
print(np.linalg.det(T1[:3, :3]))
print("========================")
print(T2)
print(np.linalg.det(T2[:3, :3]))
print("========================")
print(T3)
print(np.linalg.det(T3[:3, :3]))
print("========================")
print(T4)
print(np.linalg.det(T4[:3, :3]))
print("========================")
print(np.linalg.norm(T1-T2), np.linalg.norm(T2-T3), np.linalg.norm(T3-T4))